# 使用fastText对文本进行分类
数据源为清华大学的新闻分类文本数据集
### 第一步获取分类文本：输出数据格式： 样本 + 样本标签

In [2]:
import jieba
import os

In [3]:
basedir = "/Users/bob/Downloads/THUCNews/"
dir_list = ['affairs','constellation','economic','edu','ent','fashion','game','home','house','lottery','science','sports','society','stock']
# 生成fastext的训练和测试数据集

ftrain = open("news_fasttext_train.txt","w")
ftest = open("news_fasttext_test.txt","w")

num = -1
for e in dir_list:
    num += 1
    indir = basedir + e + '/'
    files = os.listdir(indir)
    count = 0
    for fileName in files:
        count += 1            
        filepath = indir + fileName
        with open(filepath,'r') as fr:
            text = fr.read()
        text = str(text.encode("utf-8"),'utf-8')
        seg_text = jieba.cut(text.replace("\t"," ").replace("\n"," "))
        outline = " ".join(seg_text)
        outline = outline + "\t__label__" + e + "\n"
#         print outline
#         break

        if count < 10000:
            ftrain.write(outline)
            ftrain.flush()
            continue
        elif count  < 20000:
            ftest.write(outline)
            ftest.flush()
            continue
        else:
            break

ftrain.close()
ftest.close()
print('完成输出数据！')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/rv/4n3zthdd11s0jdnqdxk070mw0000gn/T/jieba.cache
Loading model cost 0.992 seconds.
Prefix dict has been built successfully.


完成输出数据！


In [4]:
import fasttext
#训练模型
classifier = fasttext.train_supervised("news_fasttext_train.txt",label_prefix="__label__")

#load训练好的模型
#classifier = fasttext.load_model('news_fasttext.model.bin', label_prefix='__label__')
print('训练完成！')

训练完成！


In [5]:
result = classifier.test("news_fasttext_test.txt")
print('precision：', result[1])

precision： 0.8779119512388748


In [6]:
labels_right = []
texts = []
with open("news_fasttext_test.txt") as fr:
    for line in fr:
        line = str(line.encode("utf-8"), 'utf-8').rstrip()
        labels_right.append(line.split("\t")[1].replace("__label__",""))
        texts.append(line.split("\t")[0])
    #     print labels
    #     print texts
#     break
labels_predict = [term[0] for term in classifier.predict(texts)[0]] #预测输出结果为二维形式
# print labels_predict

text_labels = list(set(labels_right))
text_predict_labels = list(set(labels_predict))
print(text_predict_labels)
print(text_labels)
print()

A = dict.fromkeys(text_labels,0)  #预测正确的各个类的数目
B = dict.fromkeys(text_labels,0)   #测试数据集中各个类的数目
C = dict.fromkeys(text_predict_labels,0) #预测结果中各个类的数目
for i in range(0,len(labels_right)):
    B[labels_right[i]] += 1
    C[labels_predict[i]] += 1
    if labels_right[i] == labels_predict[i].replace('__label__', ''):
        A[labels_right[i]] += 1

print('预测正确的各个类的数目:', A) 
print()
print('测试数据集中各个类的数目:', B)
print()
print('预测结果中各个类的数目:', C)
print()
#计算准确率，召回率，F值
for key in B:
    try:
        r = float(A[key]) / float(B[key])
        p = float(A[key]) / float(C['__label__' + key])
        f = p * r * 2 / (p + r)
        print("%s:\t p:%f\t r:%f\t f:%f" % (key,p,r,f))
    except:
        print("error:", key, "right:", A.get(key,0), "real:", B.get(key,0), "predict:",C.get(key,0))

['__label__edu', '__label__science', '__label__affairs', '__label__ent', '__label__economic', '__label__lottery', '__label__constellation', '__label__society', '__label__game', '__label__home', '__label__house', '__label__stock', '__label__sports', '__label__fashion']
['sports', 'game', 'fashion', 'science', 'stock', 'economic', 'house', 'ent', 'home', 'affairs', 'edu', 'society']

预测正确的各个类的数目: {'sports': 9725, 'game': 7755, 'fashion': 3188, 'science': 9044, 'stock': 8804, 'economic': 8716, 'house': 9522, 'ent': 9113, 'home': 7877, 'affairs': 7271, 'edu': 9316, 'society': 9197}

测试数据集中各个类的数目: {'sports': 10000, 'game': 10000, 'fashion': 3369, 'science': 10000, 'stock': 10000, 'economic': 10000, 'house': 10000, 'ent': 10000, 'home': 10000, 'affairs': 10000, 'edu': 10000, 'society': 10000}

预测结果中各个类的数目: {'__label__edu': 10103, '__label__science': 12166, '__label__affairs': 7496, '__label__ent': 9877, '__label__economic': 9228, '__label__lottery': 458, '__label__constellation': 10, '__labe